In [213]:
import numpy as np

def round_over(state):
    row_sums = np.abs(np.sum(state, axis=0))
    col_sums = np.abs(np.sum(state, axis=1))
    first_trace = np.abs(np.trace(state))
    second_trace = np.abs(np.trace(state[::-1]))
    if np.any(row_sums == 3) or np.any(col_sums == 3) or first_trace == 3 or second_trace == 3:
        return True, 'win'
    elif np.all(state):
        return True, 'draw'
    else:
        return False, None
    
def get_reward(state, player_marker):
    row_sums = np.sum(state, axis=0) * player_marker
    col_sums = np.sum(state, axis=1) * player_marker
    first_trace = np.trace(state) * player_marker
    second_trace = np.trace(state[::-1]) * player_marker
    if np.any(row_sums == 3) or np.any(col_sums == 3) or first_trace == 3 or second_trace == 3:
        return 1
    elif np.any(row_sums == -3) or np.any(col_sums == -3) or first_trace == -3 or second_trace == -3:
        return -1
    else:
        return 0
    
def play_round(player_q_table_lookup, epsilon, epsilon_decay):
    state = np.zeros((3,3), dtype='int8')
    state_list = [state]
    action_list = []
    stop_game = False
    learning_rate = 0.5
    
    player_list = ['player1', 'player2']
    marker_list = [1, -1]
    turn = 0
    while not stop_game:
        player_idx = turn%2
        waiter_idx = (turn+1)%2
        player = player_list[player_idx]
        waiter = player_list[waiter_idx]
        
        # Update Q table now that we know what the previous player did
        if len(state_list) > 2:
            previous_state = state_list[-3] # Previous state Player encountered
            current_state = state_list[-1] # Current state
            action = action_list[-2] # Players previous action
            previous_q_table = get_q_table(previous_state, player_q_table_lookup[player])
            current_q_table = get_q_table(current_state, player_q_table_lookup[player])
            q_value = previous_q_table[action]
            reward = get_reward(current_state, marker_list[player_idx])
            q_value = q_value + learning_rate*(reward + np.nanmax(current_q_table) - q_value)
            previous_q_table[action] = q_value
        
        # Player makes move
        q_table_lookup = player_q_table_lookup[player]
        state, q_table_lookup, action = make_move(state, q_table_lookup, marker_list[player_idx], epsilon)
        
        state_list.append(state)
        action_list.append(action)
        stop_game, round_over_type = round_over(state)
        
        turn += 1
        epsilon *= epsilon_decay
    if round_over_type is 'win':
        print("{} wins".format(player))
        reward = 1
    else:
        print("Draw")
        reward = 0
    # Update for the winner (Player)
    q_table = get_q_table(state_list[-2], player_q_table_lookup[player])
    q_table[action_list[-1]] = reward
    q_table = get_q_table(state_list[-3], player_q_table_lookup[waiter])
    q_table[action_list[-2]] = -reward
    
    return epsilon
    
def get_q_table(state, q_table_lookup):
    state_key = str(state)
    if state_key in q_table_lookup:
        q_table = q_table_lookup[state_key]['q']
    else:
        q_table = np.where(state == 0, state, np.NaN) # Set NaN where we have pieces placed
        q_table_lookup[state_key] = {}
        q_table_lookup[state_key]['q'] = q_table
        q_table_lookup[state_key]['state'] = state.copy()
    return q_table

def make_move(state, q_table_lookup, marker, epsilon):
    state = state.copy() # Very important to make a copy here so we dont change in all state arrays
    q_table = get_q_table(state, q_table_lookup)
    if np.random.rand() < epsilon:
        viable_actions = np.argwhere(~np.isnan(q_table))
    else:
        viable_actions = np.argwhere(q_table == np.nanmax(q_table))
    action = tuple(viable_actions[np.random.randint(len(viable_actions))])
    state[action] = marker
    return state, q_table_lookup, action
    
    

# Initilize Q-tables for the two players
player_q_table_lookup = {
    'player1': {},
    'player2': {}
}

n_rounds = 15000
epsilon = 0.9
epsilon_decay = 0.9995

for i in range(n_rounds):
    epsilon = play_round(player_q_table_lookup, epsilon, epsilon_decay)
    

player2 wins
player2 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
Draw
player2 wins
player2 wins
player2 wins
player2 wins
player1 wins
player1 wins
Draw
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player2 wins
player2 wins
player1 wins
player2 wins
player1 wins
Draw
player1 wins
player1 wins
player1 wins
player2 wins
player1 wins
player1 wins
player2 wins
player2 wins
Draw
player1 wins
player1 wins
player2 wins
player1 wins
player1 wins
player1 wins
player1 wins
player2 wins
Draw
player1 wins
player2 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player1 wins
player2 wins
player2 wins
player2 wins
player1 wins
player2 wins
player1 wins
player1 wins
player1 wins
player1 wins
Draw
player1 wins
player2 wins
player1 wins
player1 wins
player1 wins
Draw
Draw
player2 wins
player1 wins
player2 wins
player1 wins
player1 win

Draw
Draw
player1 wins
Draw
player1 wins
player1 wins
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
player2 wins
player1 wins
Draw
Draw
player1 wins
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
player1 wins
Draw
Draw
Draw
Draw
player1 wins
Draw
Draw
Draw
player1 wins
Draw
Draw
Draw
player1 wins
Draw
Draw
player1 wins
player2 wins
Draw
Draw
player1 wins
player1 wins
Draw
Draw
player2 wins
player1 wins
player1 wins
Draw
Draw
player2 wins
Draw
player1 wins
Draw
player2 wins
player1 wins
Draw
Draw
player1 wins
Draw
player2 wins
player1 wins
Draw
player1 wins
Draw
player1 wins
player2 wins
Draw
player1 wins
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
player1 wins
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
player1 wins
Draw
Draw
player1 wins
Draw
Draw
Draw
Draw
player1 wins
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
player2 wins
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
player1 wins
Draw
Draw
Dra

Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


In [210]:
def write_results(player, player_q_table_lookup):
    states = []
    q_tables = []
    for key, value in player_q_table_lookup[player].items():
        states.append(value['state'])
        q_tables.append(value['q'])
    states = tuple(states)
    q_tables = tuple(q_tables)
    states = np.concatenate(states, axis=1)
    q_tables = np.concatenate(q_tables, axis=1)

    output = ""
    for i in range(3):
        for j in range(states.shape[1]-1):
            output += str(states[i,j]) + ", "
        output += str(states[-1,-1]) + "\n"
    for i in range(3):
        for j in range(states.shape[1]-1):
            output += "{}".format(q_tables[i,j]) + ", "
        output += "{}".format(q_tables[i,j]) + "\n"
    out_file = open(player + ".csv", "w")
    out_file.write(output)
    out_file.close()
    
write_results('player1', player_q_table_lookup)
write_results('player2', player_q_table_lookup)

In [190]:
state = np.zeros((3,3), dtype='int8')
state[0,1] = 1
key = str(state)
print(player_q_table_lookup['player2'][key]['q'])

[[0.00512     nan 0.     ]
 [0.      0.      0.     ]
 [0.      0.      0.     ]]
